In [125]:
import pandas as pd
import numpy as np
from datetime import date
import datetime
import mysql.connector
from mysql.connector import Error
import os


### Unión fuentes proveedores Data

In [126]:
df_MU = pd.read_excel(r"C:\Users\luis.montoya\Downloads\Mostrador\MUSemanal.xlsx")
usuarios_MU = pd.read_excel(r"C:\Users\luis.montoya\Downloads\Mostrador\Puntos MU.xlsx")
df_MU=df_MU[["uuid","num_orden","fecha_inicio","valor_total_servicio","tipo_pago","cant_paradas","distancia_km","estado_servicio","descripcion","iniciado","asignado","llego_punto","salio_punto","llego_cliente","finalizado","entrega_fallida","tipificación_fallida","nombre_solicitante"]]

df_MU['iniciado'] = pd.to_datetime(df_MU.iniciado).dt.tz_localize(None)
df_MU['asignado'] = pd.to_datetime(df_MU.asignado).dt.tz_localize(None)
df_MU['llego_punto'] = pd.to_datetime(df_MU.llego_punto).dt.tz_localize(None)
df_MU['salio_punto'] = pd.to_datetime(df_MU.salio_punto).dt.tz_localize(None)
df_MU['llego_cliente'] = pd.to_datetime(df_MU.llego_cliente).dt.tz_localize(None)
df_MU['finalizado'] = pd.to_datetime(df_MU.finalizado).dt.tz_localize(None)

    
df_MU["iniciado"] = pd.to_datetime(df_MU["iniciado"],format="%Y/%m/%d %H:%M:%S")
df_MU["asignado"] = pd.to_datetime(df_MU["asignado"],format="%Y/%m/%d %H:%M:%S")
df_MU["llego_punto"] = pd.to_datetime(df_MU["llego_punto"],format="%Y/%m/%d %H:%M:%S")
df_MU["salio_punto"] = pd.to_datetime(df_MU["salio_punto"],format="%Y/%m/%d %H:%M:%S")
df_MU["llego_cliente"] = pd.to_datetime(df_MU["llego_cliente"],format="%Y/%m/%d %H:%M:%S")
df_MU["finalizado"] = pd.to_datetime(df_MU["finalizado"],format="%Y/%m/%d %H:%M:%S")

df_MU["Proveedor"] = "MU"

df_MU = pd.merge(df_MU,usuarios_MU,left_on='nombre_solicitante',right_on='nombre_solicitante',how = 'left')
df_MU["Marcacion"] = " "
df_MU["Id_origen"] = " "

fechas_inicioMU = df_MU["fecha_inicio"]
nuevas_fechas = []

for elemento in fechas_inicioMU:
    elemento = elemento[0:10]
    nuevas_fechas.append(elemento)

df_MU["fecha_inicio"] = nuevas_fechas

df_MU["fecha_inicio"] = pd.to_datetime(df_MU["fecha_inicio"],format = "%Y-%m-%d")


df_MU  = df_MU.reindex(columns = ['uuid','fecha_inicio','cant_paradas','distancia_km','entrega_fallida','tipificación_fallida','descripcion','iniciado', 'asignado', 'llego_punto', 'salio_punto','llego_cliente', 'finalizado','num_orden','tipo_pago','Id_origen','Marcacion','valor_total_servicio','Cod','Proveedor','estado_servicio'])
df_MU.columns = ['uuid','fecha_inicio','cant_paradas','distancia_km','Finalizado_Fallido','Tipo_Fallido','descripcion','iniciado','asignado','llego_punto','salio_punto','llego_cliente','finalizado','numorden','cliente_final_tipo_pago','ID_Origen','Marcacion_Real','Valor_con_Descuentos','ID_Solicitante','Proveedor','Estado']

df_MU.loc[df_MU["Finalizado_Fallido"]=="Sí","Finalizado_Fallido"] = "Si"

In [127]:
df_QuickGo = pd.DataFrame(columns = ['ID Servicio','Total (Km)','Precio Total','Número de orden','Fecha de solicitud','Fecha de creación','Email','Cantidad Paradas','Estado','Fecha Hora Asignado','Fecha Hora Llegada Primera Parada','Fecha Hora Salida Primera Parada','Fecha Hora Llegada Segunda Parada','Fecha Hora Finalizacion Servicio','Tipo de Novedad'])
for the_file in os.listdir(r"C:\Users\luis.montoya\Downloads\Mostrador\Quick Go"):
    nombre_archivo = os.path.join(r"C:\Users\luis.montoya\Downloads\Mostrador\Quick Go",the_file)
    QuickGo_parcial = pd.read_csv(nombre_archivo,sep = ";")
    QuickGo_parcial = QuickGo_parcial[['ID Servicio','Total (Km)','Precio Total','Número de orden','Fecha de solicitud','Fecha de creación','Email','Cantidad Paradas','Estado','Fecha Hora Asignado','Fecha Hora Llegada Primera Parada','Fecha Hora Salida Primera Parada','Fecha Hora Llegada Segunda Parada','Fecha Hora Finalizacion Servicio','Tipo de Novedad']]
    df_QuickGo = pd.concat([df_QuickGo,QuickGo_parcial],axis  = 0)

df_QuickGo.columns = ['ID_Servicio','Total_(Km)','Precio_Total','Número_orden','fecha_solicitud','fecha_creacion','Email','Cantidad_Paradas','Estado','fecha_asignado','llegada_punto','salida_punto','llegada_cliente','fecha_finalizado','Tipo_Novedad']

C:\Users\luis.montoya\AppData\Local\Temp\ipykernel_19380\562936005.py:4: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  QuickGo_parcial = pd.read_csv(nombre_archivo,sep = ";")


In [128]:
df_QuickGo['fecha_solicitud'] = pd.to_datetime(df_QuickGo['fecha_solicitud'],format="%d/%m/%Y %H:%M:%S")
df_QuickGo['fecha_asignado'] = pd.to_datetime(df_QuickGo['fecha_asignado'],format="%Y/%m/%d %H:%M:%S")
df_QuickGo['llegada_punto'] = pd.to_datetime(df_QuickGo['llegada_punto'],format="%Y/%m/%d %H:%M:%S")
df_QuickGo['salida_punto'] = pd.to_datetime(df_QuickGo['salida_punto'],format="%Y/%m/%d %H:%M:%S")
df_QuickGo['llegada_cliente'] = pd.to_datetime(df_QuickGo['llegada_cliente'],format="%Y/%m/%d %H:%M:%S")
df_QuickGo['fecha_finalizado'] = pd.to_datetime(df_QuickGo['fecha_finalizado'],format="%Y/%m/%d %H:%M:%S")

df_QuickGo['fecha_solicitud'] = pd.to_datetime(df_QuickGo.fecha_solicitud).dt.tz_localize(None)
df_QuickGo['fecha_asignado'] = pd.to_datetime(df_QuickGo.fecha_asignado).dt.tz_localize(None)
df_QuickGo['llegada_punto'] = pd.to_datetime(df_QuickGo.llegada_punto).dt.tz_localize(None)     
df_QuickGo['salida_punto'] = pd.to_datetime(df_QuickGo.salida_punto).dt.tz_localize(None)
df_QuickGo['llegada_cliente'] = pd.to_datetime(df_QuickGo.llegada_cliente).dt.tz_localize(None) 
df_QuickGo['fecha_finalizado'] = pd.to_datetime(df_QuickGo.fecha_finalizado).dt.tz_localize(None) 

df_QuickGo["Proveedor"] = "Quick Go"
df_QuickGo ["Marcacion"] = " "
df_QuickGo["Id_origen"] = " "
df_QuickGo["descripcion"] = " "
df_QuickGo["tipo_pago"] = np.nan

fechas_creacionGo = df_QuickGo["fecha_creacion"]
nuevas_fechas = []

for elemento in fechas_creacionGo:
    elemento = elemento[0:10]
    nuevas_fechas.append(elemento)

df_QuickGo["fecha_creacion"] = nuevas_fechas
df_QuickGo["fecha_creacion"] = pd.to_datetime(df_QuickGo["fecha_creacion"],format="%Y-%m-%d")


df_QuickGo["Finalizado_Fallido"] = "Si"
df_QuickGo.loc[df_QuickGo.Tipo_Novedad.isnull(),"Finalizado_Fallido"] = "No"   

usuarios_Go =df_QuickGo["Email"]
Id_solicitante = []
for elemento in usuarios_Go:
    elemento = elemento[elemento.find('.') + 1:elemento.find('@')]
    Id_solicitante.append(elemento)

df_QuickGo["Id_solicitante"] = Id_solicitante

df_QuickGo = df_QuickGo.drop(['Email'],axis=1)

df_QuickGo = df_QuickGo.reindex(columns=['ID_Servicio','fecha_creacion','Cantidad_Paradas','Total_(Km)','Finalizado_Fallido','Tipo_Novedad','descripcion','fecha_solicitud','fecha_asignado','llegada_punto','salida_punto','llegada_cliente','fecha_finalizado','Número_orden','tipo_pago','Id_origen','Marcacion','Precio_Total','Id_solicitante','Proveedor','Estado'])
df_QuickGo.columns = ['uuid','fecha_inicio','cant_paradas','distancia_km','Finalizado_Fallido','Tipo_Fallido','descripcion','iniciado','asignado','llego_punto','salio_punto','llego_cliente','finalizado','numorden','cliente_final_tipo_pago','ID_Origen','Marcacion_Real','Valor_con_Descuentos','ID_Solicitante','Proveedor','Estado']




In [129]:
df_QuickHelp = pd.read_excel(r"C:\Users\luis.montoya\Downloads\Mostrador\QuickHelpSemanal.xlsx")
df_QuickHelp=df_QuickHelp[['estado', 'entrega','fecha_entrega','usuario_creador','fecha_creacion','fecha_aceptacion','fecha_llegada','fecha_cierre','tipo_servicio','factura_cv']]
df_QuickHelp["Finalizado_Fallido"] = "No"
df_QuickHelp["Tipo_Novedad"] = np.nan
df_QuickHelp["Cant_paradas"] = np.nan
df_QuickHelp["Total_(km)"] =np.nan
df_QuickHelp["llegada_punto"] =np.nan
df_QuickHelp["llegada_punto"] = pd.to_datetime(df_QuickHelp["llegada_punto"],format="%Y/%m/%d %H:%M:%S")
df_QuickHelp["salida_punto"] = np.nan
df_QuickHelp["salida_punto"] = pd.to_datetime(df_QuickHelp["salida_punto"],format="%Y/%m/%d %H:%M:%S")
df_QuickHelp["tipo_pago"] = np.nan
df_QuickHelp["Id_origen"] = " "
df_QuickHelp["Marcacion"] = " "
df_QuickHelp["Precio_total"] = np.nan
df_QuickHelp["Proveedor"] = "Quick Help"

usuarios_Help = df_QuickHelp["usuario_creador"]
Id_solicitante = []
for elemento in usuarios_Help:
    elemento = elemento[elemento.find('.') + 1 : elemento.find('@')]
    Id_solicitante.append(elemento)

df_QuickHelp["Id_solicitante"] = Id_solicitante

df_QuickHelp = df_QuickHelp.reindex(columns = ['entrega','fecha_entrega','Cant_paradas','Total_(km)','Finalizado_Fallido','Tipo_Novedad','tipo_servicio','fecha_creacion','fecha_aceptacion','llegada_punto','salida_punto','fecha_llegada','fecha_cierre','factura_cv','tipo_pago','Id_origen','Marcacion','Precio_total','Id_solicitante','Proveedor','estado'])
df_QuickHelp.columns = ['uuid','fecha_inicio','cant_paradas','distancia_km','Finalizado_Fallido','Tipo_Fallido','descripcion','iniciado','asignado','llego_punto','salio_punto','llego_cliente','finalizado','numorden','cliente_final_tipo_pago','ID_Origen','Marcacion_Real','Valor_con_Descuentos','ID_Solicitante','Proveedor','Estado']                          


In [130]:
df_Pibox = pd.read_excel(r"C:\Users\luis.montoya\Downloads\Mostrador\PiboxSemanal.xlsx")
usuarios_Pibox = pd.read_excel(r"C:\Users\luis.montoya\Downloads\Mostrador\Puntos Pibox.xlsx")
df_Pibox=df_Pibox[['UUID','Orden Parada','Fecha Inicio','Distancia (Km)','Estado','Finalizado Fallido','Descripción Servicio Fallido','Iniciado','Asignado','Llegó al Origen','Salió de Origen','Llegó donde el Cliente','Finalizó Servicio','Num. Orden','Nombre Usuario','Valor Total Servicio']]
df_Pibox["tipo_pago"] = np.nan
df_Pibox["Id_origen"] = " "
df_Pibox["Marcacion"] = " "
df_Pibox["descripcion"] = " "
df_Pibox["Proveedor"] = "Pibox"

df_Pibox = pd.merge(df_Pibox,usuarios_Pibox,left_on='Nombre Usuario',right_on='Nombre Usuario',how='left')

df_Pibox = df_Pibox.reindex(columns = ['UUID','Fecha Inicio','Orden Parada','Distancia (Km)','Finalizado Fallido','Descripción Servicio Fallido','descripcion','Iniciado','Asignado','Llegó al Origen','Salió de Origen','Llegó donde el Cliente','Finalizó Servicio','Num. Orden','tipo_pago','Id_origen','Marcacion','Valor Total Servicio','Cod','Proveedor','Estado'])
df_Pibox.columns = ['uuid','fecha_inicio','cant_paradas','distancia_km','Finalizado_Fallido','Tipo_Fallido','descripcion','iniciado','asignado','llego_punto','salio_punto','llego_cliente','finalizado','numorden','cliente_final_tipo_pago','ID_Origen','Marcacion_Real','Valor_con_Descuentos','ID_Solicitante','Proveedor','Estado']                          

ordenes = df_Pibox["numorden"]
numorden_pibox_2 = []
for element in ordenes:
    element = element.replace('PEDIDO ','')
    numorden_pibox_2.append(element)

df_Pibox['numorden'] = numorden_pibox_2

df_Pibox.loc[df_Pibox["Estado"]=="Expirado","Estado"] ="Cancelado"

distancias = df_Pibox["distancia_km"]
distancias_nuevo = []
for elemento in distancias: 
    elemento = elemento[0:elemento.find(",")]
    distancias_nuevo.append(elemento)
df_Pibox["distancia_km"] = distancias_nuevo
df_Pibox["distancia_km"] = pd.to_numeric(df_Pibox["distancia_km"],downcast ="integer")

df_Pibox["Valor_con_Descuentos"] = df_Pibox["Valor_con_Descuentos"].fillna("$")
plata = df_Pibox["Valor_con_Descuentos"]
plata2 = []
for registro in plata:
    valorn = registro.replace("$","")
    valorn = valorn.replace(".","")
    
    plata2.append(valorn)
    
df_Pibox["Valor_con_Descuentos"] = plata2
df_Pibox["Valor_con_Descuentos"] = pd.to_numeric(df_Pibox["Valor_con_Descuentos"],downcast="integer")

In [131]:
df_Oficios = pd.read_excel(r"C:\Users\luis.montoya\Downloads\Mostrador\OficiosSemanal.xlsx")
usuarios_Oficios = pd.read_excel(r"C:\Users\luis.montoya\Downloads\Mostrador\Puntos Oficios.xlsx")
df_Oficios=df_Oficios[['Service','Fecha Servicio','Status','Km','# Paradas','Valor total servicio','Hs-en espera','Hs-asignado','Hs-Llegando/origen','Hs-saliendo/origen','Hs-llegando/destino','Hs-Finalizado','id-Cliente','Convenio','Punto','Origen']]
df_Oficios.columns =['Service','fecha_servicio','Status','Km','Paradas','Valor_servicio','en_espera','asignado','Llegando_origen','saliendo_origen','llegando_destino','Finalizado','id_cliente','Convenio','Punto','Origen']

def recorte_fecha(d_frame,df_Oficios):
    
    d_frame.columns = ['Service','fecha_origen']
    d_frame = d_frame[~d_frame.fecha_origen.isnull()]
    lista = d_frame["fecha_origen"]
    cant_columns = len(df_Oficios.columns)
    
    resultado = []
    for elemento in lista:
        elemento = elemento[0:20]
        resultado.append(elemento)
    
    d_frame["fecha_origen"] = resultado
    d_frame["fecha_origen"] = pd.to_datetime(d_frame["fecha_origen"],format="%Y/%m/%d %H:%M:%S")
    
    df_Oficios = pd.merge(df_Oficios,d_frame,left_on="Service",right_on="Service",how="left")
    
    
    return df_Oficios

fechas = ['en_espera','asignado','Llegando_origen','saliendo_origen','llegando_destino','Finalizado']

for reg in fechas:   
    df_Oficios_fechas  = df_Oficios[['Service',reg]]    
    df_Oficios = recorte_fecha(df_Oficios_fechas,df_Oficios)


df_Oficios.columns= ['Service','fecha_servicio', 'Status', 'Km', 'Paradas', 'Valor_servicio', 'en_espera',
       'asignado', 'Llegando_origen', 'saliendo_origen', 'llegando_destino',
       'Finalizado', 'id_cliente', 'Convenio', 'Punto', 'Origen',
       'en_espera(2)', 'asignado(2)', 'llegando_origen(2)', 'saliendo_origen(2)',
       'llegando_destino(2)', 'finalizado(2)']

df_Oficios = df_Oficios[['Service','fecha_servicio', 'Status', 'Km', 'Paradas', 'Valor_servicio','id_cliente', 'Convenio', 'Punto', 'Origen',
       'en_espera(2)', 'asignado(2)', 'llegando_origen(2)', 'saliendo_origen(2)',
       'llegando_destino(2)', 'finalizado(2)']]

fechas_oficios = df_Oficios["fecha_servicio"]
fechas_of_nuevas = []

for elemento in fechas_oficios:
    elemento = elemento[0:10]
    fechas_of_nuevas.append(elemento)

df_Oficios["fecha_servicio"] = fechas_of_nuevas
df_Oficios["fecha_servicio"] = pd.to_datetime(df_Oficios["fecha_servicio"],format="%Y-%m-%d")

df_Oficios["tipo_pago"] = np.nan
df_Oficios["Id_origen"] = " "
df_Oficios["Marcacion"] = " "
df_Oficios["Proveedor"] ="Oficios"
df_Oficios["Finalizado Fallido"] = "No"
df_Oficios["Tipo_Novedad"] = np.nan

df_Oficios=pd.merge(df_Oficios,usuarios_Oficios,left_on='Punto',right_on='Punto',how='left')

    
df_Oficios = df_Oficios.reindex(columns = ['Service','fecha_servicio','Paradas','Km','Finalizado Fallido','Tipo_Novedad','Convenio','en_espera(2)', 'asignado(2)', 'llegando_origen(2)', 'saliendo_origen(2)','llegando_destino(2)', 'finalizado(2)','id_cliente','tipo_pago','Id_origen','Marcacion','Valor_servicio','Cod','Proveedor','Status'])
df_Oficios.columns = ['uuid','fecha_inicio','cant_paradas','distancia_km','Finalizado_Fallido','Tipo_Fallido','descripcion','iniciado','asignado','llego_punto','salio_punto','llego_cliente','finalizado','numorden','cliente_final_tipo_pago','ID_Origen','Marcacion_Real','Valor_con_Descuentos','ID_Solicitante','Proveedor','Estado']                          


C:\Users\luis.montoya\AppData\Local\Temp\ipykernel_19380\1263532801.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d_frame["fecha_origen"] = resultado
C:\Users\luis.montoya\AppData\Local\Temp\ipykernel_19380\1263532801.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d_frame["fecha_origen"] = pd.to_datetime(d_frame["fecha_origen"],format="%Y/%m/%d %H:%M:%S")
C:\Users\luis.montoya\AppData\Local\Temp\ipykernel_19380\1263532801.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of 

In [132]:
df_Servicourier = pd.read_excel(r"C:\Users\luis.montoya\Downloads\Mostrador\ServicourierSemanal.xlsx")
df_Servicourier =df_Servicourier[[' Id ',' Pedido ',' Zona ',' ModoPago ',' Estado ',' EntDom ',' EntCliente ',' KMS ']]
df_Servicourier.columns = ['Id','Pedido','Zona','ModoPago','Estado','EntDom','EntCliente','KMS']
df_Servicourier = df_Servicourier[~df_Servicourier.Zona.isnull()]

df_Servicourier["Finalizado_Fallido"] = "No"
df_Servicourier["Tipo_Novedad"] = np.nan
df_Servicourier["descripcion"] = " "
df_Servicourier["Cant_paradas"] = np.nan
df_Servicourier["llegada_punto"] =np.nan
df_Servicourier["llegada_punto"] = pd.to_datetime(df_Servicourier["llegada_punto"],format="%Y/%m/%d %H:%M:%S")
df_Servicourier["salida_punto"] = np.nan
df_Servicourier["salida_punto"] = pd.to_datetime(df_Servicourier["salida_punto"],format="%Y/%m/%d %H:%M:%S")
df_Servicourier["asignacion"] = np.nan
df_Servicourier["asignacion"] = pd.to_datetime(df_Servicourier["asignacion"],format="%Y/%m/%d %H:%M:%S")
df_Servicourier["finalizado"] = df_Servicourier["EntCliente"]
df_Servicourier["Id_origen"] = " "
df_Servicourier["Marcacion"] = " "
df_Servicourier["Precio_total"] = np.nan
df_Servicourier["Proveedor"] = "Servicourier"

solicitantes_Servicourier=df_Servicourier["Zona"]
Id_solicitante = []
for elemento in solicitantes_Servicourier:
    elemento = elemento[0:3]
    Id_solicitante.append(elemento)
    
df_Servicourier["Id_solicitante"] = Id_solicitante

fechas_convertir = df_Servicourier["EntDom"]

fecha_servicio =[]

for dato in fechas_convertir:
    dato=dato.strftime("%Y-%m-%d")
    fecha_servicio.append(dato)
    
df_Servicourier["fecha_servicio"]=fecha_servicio
df_Servicourier["fecha_servicio"] = pd.to_datetime(df_Servicourier["fecha_servicio"],format="%Y-%m-%d")

df_Servicourier = df_Servicourier[['Id', 'Pedido', 'ModoPago', 'Estado', 'EntDom', 'EntCliente','KMS', 'Finalizado_Fallido', 'Tipo_Novedad', 'Cant_paradas','llegada_punto', 'salida_punto', 'asignacion', 'finalizado','Id_origen', 'Marcacion', 'Precio_total', 'Proveedor', 'Id_solicitante','fecha_servicio']]
df_Servicourier = df_Servicourier.reindex(columns=['Id','fecha_servicio','Cant_paradas','KMS','Finalizado_Fallido','Tipo_Novedad','descripcion','EntDom','asignacion','llegada_punto','salida_punto','EntCliente','finalizado','Pedido','ModoPago','Id_origen','Marcacion','Precio_total','Id_solicitante','Proveedor','Estado'])
df_Servicourier.columns = ['uuid','fecha_inicio','cant_paradas','distancia_km','Finalizado_Fallido','Tipo_Fallido','descripcion','iniciado','asignado','llego_punto','salio_punto','llego_cliente','finalizado','numorden','cliente_final_tipo_pago','ID_Origen','Marcacion_Real','Valor_con_Descuentos','ID_Solicitante','Proveedor','Estado']                          
   
    
    
    


In [133]:
df_Total = pd.concat([df_MU,df_QuickGo,df_QuickHelp,df_Pibox,df_Oficios,df_Servicourier],axis=0)
#df_Total["distancia_km"] = pd.to_numeric(df_Total["distancia_km"],downcast="float")
#df_Total["Valor_con_Descuentos"] = pd.to_numeric(df_Total["Valor_con_Descuentos"],downcast = "float")
estados_traza = ["CANCELACION","Cancelado","CERRADO CON NOVEDAD","Devuelta","Devuelto","ENTREGADO","Exitosa","Finalizado"]
df_Total = df_Total[df_Total.Estado.isin(estados_traza)]
df_Total = df_Total.loc[~((df_Total["descripcion"].str.contains("Disp[a-z]*")) |(df_Total["descripcion"].str.contains("disp[a-z]*")) | (df_Total["descripcion"].str.contains("DISP[a-z]*")))]
df_Total = df_Total[~df_Total.ID_Solicitante.isnull()]
df_Total = df_Total[df_Total["ID_Solicitante"]!="cardenas"]

In [134]:
cnx = mysql.connector.connect(
    host="172.18.50.9",
    port=3306,
    user="consultapedidos",
    password="#Cr9zV3rd3*",
    db="domicilios")



In [135]:
%%time
periodo = 202212

cur = cnx.cursor()
cur.execute("SELECT pedido, local_n, periodo FROM domi_pedidos WHERE domi_pedidos.periodo >= %s", (periodo,))

data =cur.fetchall()
Domifacil = pd.DataFrame(data)
Domifacil.columns = cur.column_names

CPU times: total: 219 ms
Wall time: 402 ms


### Cruces Marcación

In [136]:
ordenes_convert =df_Total["numorden"] 
ordenes_conver_nueva =[]
for elemento in ordenes_convert:
    elemento = str(elemento)
    ordenes_conver_nueva.append(elemento)
    
df_Total["numorden"] = ordenes_conver_nueva
    

In [137]:
df_Total_OMS = df_Total.loc[(df_Total["numorden"].str.contains("CO1"))|(df_Total["numorden"].str.contains("COCC")) |(df_Total["numorden"].str.contains("Y10"))]
df_Total_Domi = df_Total.loc[(~df_Total["numorden"].str.contains('CO|Y1',regex = True))&(df_Total["numorden"].str.contains('102|000102',regex=True))]
df_Total_Traslado = df_Total.loc[(df_Total["numorden"].str.contains("SMT"))|(df_Total["numorden"].str.contains("smt"))]

recorte = []
ordenes =df_Total_Domi["numorden"]
for orden in ordenes:
    orden = orden[orden.find("1"):20]
    orden = orden[0:9]
    recorte.append(orden)
df_Total_Domi["numorden"] = recorte


Domi_pedido = Domifacil["pedido"]
pedido2= []
for pedido in Domi_pedido:
    pedido = str(pedido)
    pedido2.append(pedido)
Domifacil["pedido"] = pedido2

df_Total_Domi = pd.merge(df_Total_Domi,Domifacil,left_on="numorden",right_on="pedido",how = "left")

df_Total_Domi = df_Total_Domi[['uuid','fecha_inicio','cant_paradas','distancia_km','Finalizado_Fallido','Tipo_Fallido','descripcion','iniciado','asignado','llego_punto','salio_punto','llego_cliente','finalizado','numorden','cliente_final_tipo_pago','ID_Origen','Marcacion_Real','Valor_con_Descuentos','ID_Solicitante','Proveedor','Estado','pedido']]
df_Total_Domi.loc[~df_Total_Domi.pedido.isnull(),"Marcacion_Real"] = "Pedido Call Center"
df_Total_Domi.loc[df_Total_Domi.pedido.isnull(),"Marcacion_Real"] = "Sin Clasificar"
 
df_Total_OMS.loc[df_Total_OMS["numorden"].str.contains("CO1"),"Marcacion_Real"] = "OMS - Ecomm"
df_Total_OMS.loc[df_Total_OMS["numorden"].str.contains("COCC"),"Marcacion_Real"] = "OMS - Call"
df_Total_OMS.loc[df_Total_OMS["numorden"].str.contains("Y10"),"Marcacion_Real"] = "OMS - Dev"
#############################################################################################33333
Traslados_Bopos = pd.read_excel(r"C:\Users\luis.montoya\Downloads\Mostrador\Traslados.xlsx")

traslado = df_Total_Traslado["numorden"]
traslado_nuevo = []
for solicitud in traslado:
    solicitud = str(solicitud)
    solicitud=solicitud.replace("smt","SMT")
    solicitud=solicitud.replace("-","")
    solicitud=solicitud.replace(" ","")
    solicitud=solicitud.replace("/","")
    solicitud=solicitud.replace("_","")
    traslado_nuevo.append(solicitud)
    
df_Total_Traslado["numorden"] = traslado_nuevo

df_Total_Traslado = pd.merge(df_Total_Traslado,Traslados_Bopos,left_on="numorden",right_on="Documento Solicitud",how = "left")

df_Total_Traslado = df_Total_Traslado[['uuid','fecha_inicio','cant_paradas','distancia_km','Finalizado_Fallido','Tipo_Fallido','descripcion','iniciado','asignado','llego_punto','salio_punto','llego_cliente','finalizado','numorden','cliente_final_tipo_pago','ID_Origen','Marcacion_Real','Valor_con_Descuentos','ID_Solicitante','Proveedor','Estado','Documento Solicitud']]

df_Total_Traslado.loc[df_Total_Traslado["Documento Solicitud"].isnull(),"Marcacion_Real"] = "Sin Clasificar - Traslado"
df_Total_Traslado.loc[~df_Total_Traslado["Documento Solicitud"].isnull(),"Marcacion_Real"] = "Traslado"

df_Total_Domi = df_Total_Domi[['uuid','fecha_inicio','cant_paradas','distancia_km','Finalizado_Fallido','Tipo_Fallido','descripcion','iniciado','asignado','llego_punto','salio_punto','llego_cliente','finalizado','numorden','cliente_final_tipo_pago','ID_Origen','Marcacion_Real','Valor_con_Descuentos','ID_Solicitante','Proveedor','Estado']]
df_Total_OMS = df_Total_OMS[['uuid','fecha_inicio','cant_paradas','distancia_km','Finalizado_Fallido','Tipo_Fallido','descripcion','iniciado','asignado','llego_punto','salio_punto','llego_cliente','finalizado','numorden','cliente_final_tipo_pago','ID_Origen','Marcacion_Real','Valor_con_Descuentos','ID_Solicitante','Proveedor','Estado']]
df_Total_Traslado = df_Total_Traslado[['uuid','fecha_inicio','cant_paradas','distancia_km','Finalizado_Fallido','Tipo_Fallido','descripcion','iniciado','asignado','llego_punto','salio_punto','llego_cliente','finalizado','numorden','cliente_final_tipo_pago','ID_Origen','Marcacion_Real','Valor_con_Descuentos','ID_Solicitante','Proveedor','Estado']]

df_Total_Cruzados = pd.concat([df_Total_Domi,df_Total_OMS,df_Total_Traslado],axis=0)
#############################################################################################################

df_Total = pd.merge(df_Total,df_Total_Cruzados,left_on="uuid",right_on="uuid",how = "left")

df_Total = df_Total[['uuid','fecha_inicio_x','cant_paradas_x','distancia_km_x','Finalizado_Fallido_x','Tipo_Fallido_x','descripcion_x','iniciado_x','asignado_x','llego_punto_x','salio_punto_x','llego_cliente_x','finalizado_x','numorden_x','cliente_final_tipo_pago_x','ID_Origen_x','Marcacion_Real_y','Valor_con_Descuentos_x','ID_Solicitante_x','Proveedor_x','Estado_x']]
df_Total.columns = ['uuid','fecha_inicio','cant_paradas','distancia_km','Finalizado_Fallido','Tipo_Fallido','descripcion','iniciado','asignado','llego_punto','salio_punto','llego_cliente','finalizado','numorden','cliente_final_tipo_pago','ID_Origen','Marcacion_Real','Valor_con_Descuentos','ID_Solicitante','Proveedor','Estado']


df_Total_Mostrador = df_Total[df_Total.Marcacion_Real.isnull()]

################################################################################################################

Mostrador = pd.read_excel(r"C:\Users\luis.montoya\Downloads\Mostrador\Mostrador.xlsx")

facturas =Mostrador["NumeroTicket"] 
factura_nuevo = []
for elemento in facturas:
    elemento = str(elemento)
    factura_nuevo.append(elemento)
Mostrador["NumeroTicket"] = factura_nuevo

tickets = df_Total_Mostrador["numorden"]
nuevo_numorden = []

for numorden in tickets:
    numorden = str(numorden)
    numorden = numorden.replace("-","")
    numorden = numorden.replace(" ","")
    numorden = numorden.replace("_","")
    numorden = numorden.replace("/","")
    nuevo_numorden.append(numorden)
df_Total_Mostrador["numorden"] = nuevo_numorden

df_Total_Mostrador = pd.merge(df_Total_Mostrador,Mostrador, left_on = "numorden",right_on="NumeroTicket",how="left")

df_Total_Mostrador = df_Total_Mostrador[['uuid','fecha_inicio','cant_paradas','distancia_km','Finalizado_Fallido','Tipo_Fallido','descripcion','iniciado','asignado','llego_punto','salio_punto','llego_cliente','finalizado','numorden','cliente_final_tipo_pago','ID_Origen','Marcacion_Real','Valor_con_Descuentos','ID_Solicitante','Proveedor','Estado','Canal']]

df_Total_Mostrador.loc[df_Total_Mostrador.Canal.isnull(),"Marcacion_Real"] = "Sin Clasificar"


######################################################################################
for i in range(0,len(df_Total_Mostrador)):
    if df_Total_Mostrador.iloc[i,16] != "Sin Clasificar":
            df_Total_Mostrador.iloc[i,16] = df_Total_Mostrador.iloc[i,21]
    else: df_Total_Mostrador.iloc[i,16] = df_Total_Mostrador.iloc[i,16]
        
        
df_Total_Mostrador.drop(["Canal"],axis=1,inplace = True)


df_Total_final = pd.concat([df_Total_Cruzados,df_Total_Mostrador],axis=0)

df_Total_final.to_excel(r"C:\Users\luis.montoya\Downloads\Mostrador\validacion_marcacion.xlsx")

df_Total_final = df_Total_final[~df_Total_final.ID_Solicitante.isnull()]
oms_busqueda = df_Total_final[df_Total_final.ID_Solicitante.isin(["OMS","oms"]) & ~df_Total_final.Marcacion_Real.isin(["Sin Clasificar", "Sin Clasificar - Traslado"])]
oms_busqueda = oms_busqueda[["uuid","numorden","Marcacion_Real"]]


C:\Users\luis.montoya\AppData\Local\Temp\ipykernel_19380\276201073.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_Total_Domi["numorden"] = recorte
C:\Users\luis.montoya\AppData\Local\Temp\ipykernel_19380\276201073.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_Total_OMS.loc[df_Total_OMS["numorden"].str.contains("CO1"),"Marcacion_Real"] = "OMS - Ecomm"
C:\Users\luis.montoya\AppData\Local\Temp\ipykernel_19380\276201073.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the document

In [138]:
resumen_total = pd.DataFrame(columns=["ORDER_NO","SHIPNODE_KEY"])
for the_file in os.listdir(r"C:\Users\luis.montoya\Downloads\Mostrador\Resumen_OMS"):
    archivo_subida = os.path.join(r"C:\Users\luis.montoya\Downloads\Mostrador\Resumen_OMS",the_file)             ###Importación del archivo de resumen
    resumen_parcial=pd.read_csv(archivo_subida,sep=",")
    columnas_resumen= ["RN","ORDER_HEADER_KEY","ENTRY_TYPE","ORDER_NO","DOCUMENT_TYPE","ORDER_DATE","ORDER_TYPE","ENTRY_TYPE2","LEVEL_OF_SERVICE","CUSTOMER_PHONE_NO","EXTN_RUN_RUT_NIT","ORIGINAL_TOTAL_AMOUNT","ENTERPRISE_KEY","PAYMENT_TYPE","STATUS_NAME","EXTN_ORG_REQ_SHIP_DATE","STATUS_DATE","EXTN_ET_FULFILMENT","SHIPNODE_KEY","BLANCOI","BLANCOII","BLANCOIII","BLANCOIV"]  
    resumen_parcial.columns=columnas_resumen
    resumen_parcial=resumen_parcial.drop(0,axis=0)
    print(the_file)
    resumen_parcial=resumen_parcial[["ORDER_NO","SHIPNODE_KEY"]]
    resumen_total = pd.concat([resumen_total,resumen_parcial],axis=0)

resumen_total = resumen_total[~resumen_total.SHIPNODE_KEY.isnull()]
resumen_total = resumen_total.drop_duplicates(subset = ['ORDER_NO'])
tiendas_oms = resumen_total["SHIPNODE_KEY"]
tiendas_no_pref =[]

for elemento in tiendas_oms:
    elemento = elemento.replace("COCV_","")
    tiendas_no_pref.append(elemento)

resumen_total["SHIPNODE_KEY"] = tiendas_no_pref
resumen_total["Origen"] = "RESUMEN"

transferencias_OMS = pd.read_csv(r"C:\Users\luis.montoya\Downloads\Mostrador\Transferencias_OMS\Transferencias_Origen_Destino.csv",sep = ",")
transferencias_OMS.columns = ["RN","EXTN_SO_NUMBER","SHIPNODE_KEY","RECEIVING_NODE","ORDER_NO","BLANCOI"]
transferencias_OMS  = transferencias_OMS.drop(0,axis =0)

transferencias_OMS = transferencias_OMS[~transferencias_OMS.RECEIVING_NODE.isnull()]

tiendas_oms_tr = transferencias_OMS["RECEIVING_NODE"]
tiendas_no_pref =[]

for elemento in tiendas_oms_tr:
    elemento = elemento.replace("COCV_","")
    tiendas_no_pref.append(elemento)

transferencias_OMS["RECEIVING_NODE"] = tiendas_no_pref
transferencias_OMS = transferencias_OMS[["RECEIVING_NODE","ORDER_NO"]]
transferencias_OMS = transferencias_OMS.reindex(columns =["ORDER_NO","RECEIVING_NODE"])
transferencias_OMS["Origen"] = "EARLY TRANSFER"
transferencias_OMS.columns = ["ORDER_NO","SHIPNODE_KEY","Origen"]

OMS_unificado = pd.concat([resumen_total,transferencias_OMS],axis = 0)





Diciembre 2022(1).csv
Diciembre 2022(2).csv
Diciembre 2022(3).csv
Diciembre 2022(4).csv
Diciembre 2022(5).csv
Diciembre 2022(6).csv
Diciembre 2022(7).csv
Noviembre 2022(1).csv
Noviembre 2022(10).csv
Noviembre 2022(2).csv
Noviembre 2022(3).csv
Noviembre 2022(4).csv
Noviembre 2022(5).csv
Noviembre 2022(6).csv
Noviembre 2022(7).csv
Noviembre 2022(8).csv
Noviembre 2022(9).csv


In [139]:
oms_busqueda_co = oms_busqueda[(oms_busqueda.numorden.str.contains("CO1"))] 
oms_busqueda_cocc = oms_busqueda[(oms_busqueda.numorden.str.contains("COCC"))] 
oms_busqueda_y = oms_busqueda[(oms_busqueda.numorden.str.contains("Y10"))] 

ordenes_oms = oms_busqueda_co["numorden"]
ordenes_cruce = []
for elemento in ordenes_oms:
    elemento = elemento[elemento.find("C"):elemento.find("C") + 10]
    ordenes_cruce.append(elemento)
    
oms_busqueda_co["numorden"] = ordenes_cruce


ordenes_oms = oms_busqueda_cocc["numorden"]
ordenes_cruce = []
for elemento in ordenes_oms:
    elemento = elemento[elemento.find("C"):elemento.find("C")+12]
    ordenes_cruce.append(elemento)
    
oms_busqueda_cocc["numorden"] = ordenes_cruce 


ordenes_oms = oms_busqueda_y["numorden"]
ordenes_cruce = []
for elemento in ordenes_oms:
    elemento = elemento[elemento.find("Y"):elemento.find("Y") + 10]
    ordenes_cruce.append(elemento)
    
oms_busqueda_y["numorden"] = ordenes_cruce 

oms_busqueda_1 = pd.concat([oms_busqueda_co,oms_busqueda_cocc,oms_busqueda_y],axis=0)

oms_busqueda_1 = pd.merge(oms_busqueda_1,OMS_unificado,left_on="numorden",right_on ="ORDER_NO",how = "left")

 
oms_busqueda_1.loc[(oms_busqueda_1["Origen"]=="EARLY TRANSFER") & (oms_busqueda_1.numorden.str.contains("Y10")),"Marcacion_Real"] == "Transferencia OMS"
oms_busqueda_1.loc[(oms_busqueda_1["Origen"]=="RESUMEN") & (oms_busqueda_1.numorden.str.contains("Y10")),"Marcacion_Real"] == "Dev - OMS"
oms_busqueda_1 = oms_busqueda_1[["uuid","numorden","Marcacion_Real","SHIPNODE_KEY"]]
oms_busqueda_1.columns = ["uuid","numorden","Marcacion_Real","Tienda"]




C:\Users\luis.montoya\AppData\Local\Temp\ipykernel_19380\3491119546.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  oms_busqueda_co["numorden"] = ordenes_cruce
C:\Users\luis.montoya\AppData\Local\Temp\ipykernel_19380\3491119546.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  oms_busqueda_cocc["numorden"] = ordenes_cruce
C:\Users\luis.montoya\AppData\Local\Temp\ipykernel_19380\3491119546.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .lo

In [140]:
oms_busqueda_2 = oms_busqueda[(~oms_busqueda["numorden"].str.contains('CO|Y1',regex = True))&(oms_busqueda["numorden"].str.contains('102|000102',regex=True))]
oms_busqueda_2 = pd.merge(oms_busqueda_2,Domifacil, left_on = "numorden",right_on ="pedido",how ="left")
oms_busqueda_2 = oms_busqueda_2[["uuid","numorden","Marcacion_Real","local_n"]]
oms_busqueda_2.columns =["uuid","numorden","Marcacion_Real","Tienda"]

oms_busqueda_3 = oms_busqueda[(oms_busqueda["numorden"].str.contains("SMT"))|(oms_busqueda["numorden"].str.contains("smt"))]
oms_busqueda_3 = pd.merge(oms_busqueda_3,Traslados_Bopos,left_on ="numorden",right_on="Documento Solicitud", how = "left")
oms_busqueda_3 = oms_busqueda_3[["uuid","numorden","Marcacion_Real","Sucursal  Solitante"]]
oms_busqueda_3.columns = ["uuid","numorden","Marcacion_Real","Tienda"]

oms_busqueda_final = pd.concat([oms_busqueda_1,oms_busqueda_2,oms_busqueda_3],axis = 0)

df_Total_final = pd.merge(df_Total_final,oms_busqueda_final, left_on ="uuid",right_on ="uuid",how = "left")




In [141]:
solicitantes = df_Total_final["ID_Solicitante"]
i = 0
for elemento in solicitantes:
    if (str(elemento) == "OMS") | (str(elemento) == "oms"):
        df_Total_final.iloc[i,18] = df_Total_final.iloc[i,23]
        df_Total_final.iloc[i,16] = df_Total_final.iloc[i,22]
    
    i = i + 1

In [142]:
hoy = date.today()
anio = hoy.year
mes= hoy.month

if mes<10 :
    mes = '0' + str(mes)
else: mes = str(mes)

anio = str(anio)

df_Total_final = df_Total_final[['uuid', 'fecha_inicio', 'cant_paradas', 'distancia_km',
       'Finalizado_Fallido', 'Tipo_Fallido', 'descripcion', 'iniciado',
       'asignado', 'llego_punto', 'salio_punto', 'llego_cliente', 'finalizado',
       'numorden_x', 'cliente_final_tipo_pago', 'ID_Origen',
       'Marcacion_Real_x', 'Valor_con_Descuentos', 'ID_Solicitante',
       'Proveedor', 'Estado']]


df_Total_final = df_Total_final.drop_duplicates(subset ="uuid" )

df_Total_final.to_excel(r"C:\Users\luis.montoya\Downloads\Mostrador\TotalCruceMostrador.xlsx",index=False)

df_Total_final = df_Total_final[~df_Total_final.ID_Solicitante.isnull()]
df_Total_final = df_Total_final[df_Total_final["ID_Solicitante"]!=" "]
#df_Total_final["ID_Solicitante"] = pd.to_numeric(df_Total_final["ID_Solicitante"],downcast ="integer")


df_Total_final.columns = ['uuid','fecha_inicio','cant_paradas','distancia_km','Finalizado Fallido','Tipo Fallido','descripcion','iniciado','asignado','llego_punto','salio_punto','llego_cliente','finalizado','numorden','cliente_final_tipo_pago','2. ID Origen','16. Marcación Real','Valor con Descuentos','ID Solicitante','Proveedor','Estado']


df_Total_cancelados = df_Total_final[df_Total_final.Estado.isin(["CANCELACION","Cancelado"])]
df_Total_cancelados = df_Total_cancelados[['uuid','fecha_inicio','numorden','ID Solicitante','Proveedor']]

nombre_archivo = anio + mes + " Cancelados.xlsx"
ruta = os.path.join(r"C:\Users\luis.montoya\OneDrive - Grupo Socofar\PowerBI - Última milla\Cancelados",nombre_archivo)
df_Total_cancelados.to_excel(ruta,index=False)

df_Total_final = df_Total_final[df_Total_final.Estado.isin(["CERRADO CON NOVEDAD","Devuelta","Devuelto","ENTREGADO","Exitosa","Finalizado"])] 


df_MU_total  = df_Total_final[df_Total_final["Proveedor"] == "MU"]
df_MU_total = df_MU_total[['uuid','fecha_inicio','cant_paradas','distancia_km','Finalizado Fallido','Tipo Fallido','descripcion','iniciado','asignado','llego_punto','salio_punto','llego_cliente','finalizado','numorden','cliente_final_tipo_pago','2. ID Origen','16. Marcación Real','Valor con Descuentos','ID Solicitante']]

df_QuickGo_total  = df_Total_final[df_Total_final["Proveedor"] == "Quick Go"]
df_QuickGo_total = df_QuickGo_total[['uuid','fecha_inicio','cant_paradas','distancia_km','Finalizado Fallido','Tipo Fallido','descripcion','iniciado','asignado','llego_punto','salio_punto','llego_cliente','finalizado','numorden','cliente_final_tipo_pago','2. ID Origen','16. Marcación Real','Valor con Descuentos','ID Solicitante']]

df_QuickHelp_total  = df_Total_final[df_Total_final["Proveedor"] == "Quick Help"]
df_QuickHelp_total= df_QuickHelp_total[['uuid','fecha_inicio','cant_paradas','distancia_km','Finalizado Fallido','Tipo Fallido','descripcion','iniciado','asignado','llego_punto','salio_punto','llego_cliente','finalizado','numorden','cliente_final_tipo_pago','2. ID Origen','16. Marcación Real','Valor con Descuentos','ID Solicitante']]


df_Pibox_total  = df_Total_final[df_Total_final["Proveedor"] == "Pibox"]
df_Pibox_total= df_Pibox_total[['uuid','fecha_inicio','cant_paradas','distancia_km','Finalizado Fallido','Tipo Fallido','descripcion','iniciado','asignado','llego_punto','salio_punto','llego_cliente','finalizado','numorden','cliente_final_tipo_pago','2. ID Origen','16. Marcación Real','Valor con Descuentos','ID Solicitante']]

df_Oficios_total  = df_Total_final[df_Total_final["Proveedor"] == "Oficios"]
df_Oficios_total= df_Oficios_total[['uuid','fecha_inicio','cant_paradas','distancia_km','Finalizado Fallido','Tipo Fallido','descripcion','iniciado','asignado','llego_punto','salio_punto','llego_cliente','finalizado','numorden','cliente_final_tipo_pago','2. ID Origen','16. Marcación Real','Valor con Descuentos','ID Solicitante']]


df_Servicourier_total  = df_Total_final[df_Total_final["Proveedor"] == "Servicourier"]
df_Servicourier_total= df_Servicourier_total[['uuid','fecha_inicio','cant_paradas','distancia_km','Finalizado Fallido','Tipo Fallido','descripcion','iniciado','asignado','llego_punto','salio_punto','llego_cliente','finalizado','numorden','cliente_final_tipo_pago','2. ID Origen','16. Marcación Real','Valor con Descuentos','ID Solicitante']]

nombre_archivo = anio + mes + " Consolidado MU.xlsx"
ruta  = os.path.join(r"C:\Users\luis.montoya\OneDrive - Grupo Socofar\PowerBI - Última milla\Mensajeros Urbanos\ConsolidadoServMU",nombre_archivo)
df_MU_total.to_excel(ruta,index=False,sheet_name ="Hoja1")

nombre_archivo = anio + mes + " Consolidado Quick Go.xlsx"
ruta  = os.path.join(r"C:\Users\luis.montoya\OneDrive - Grupo Socofar\PowerBI - Última milla\Quick\Quick Go",nombre_archivo)
df_QuickGo_total.to_excel(ruta,index=False,sheet_name="Plano Quick Go")

nombre_archivo = anio + mes + " Consolidado Quick.xlsx"
ruta  = os.path.join(r"C:\Users\luis.montoya\OneDrive - Grupo Socofar\PowerBI - Última milla\Quick\ConsolidadoServQuick",nombre_archivo)
df_QuickHelp_total.to_excel(ruta,index=False, sheet_name = "Hoja1")

nombre_archivo = anio + mes + " Consolidado Picap.xlsx"
ruta  = os.path.join(r"C:\Users\luis.montoya\OneDrive - Grupo Socofar\PowerBI - Última milla\Picap\Consolidado Servicios Picap",nombre_archivo)
df_Pibox_total.to_excel(ruta,index=False, sheet_name="Hoja1")

nombre_archivo = anio + mes + " Consolidado Oficios.xlsx"
ruta  = os.path.join(r"C:\Users\luis.montoya\OneDrive - Grupo Socofar\PowerBI - Última milla\Oficios\ConsolidadoServOficios",nombre_archivo)
df_Oficios_total.to_excel(ruta,index=False,sheet_name="Hoja1")

nombre_archivo = anio + mes + " Consolidado Servicourier.xlsx"
ruta  = os.path.join(r"C:\Users\luis.montoya\OneDrive - Grupo Socofar\PowerBI - Última milla\Servicourier\ConsolidadoServSC",nombre_archivo)
df_Servicourier_total.to_excel(ruta,index=False,sheet_name = "Hoja1")

### Actualización venta

In [143]:
dia = date.today()
dia = dia.day
dia= dia - 1 


try: 
    conexion = mysql.connector.connect(
        host="172.18.50.54",
        port=3306,
        user="operaciones",
        password="operaciones3",
        db="ventas"
    
    )
    
    if conexion.is_connected(): 
        print("Conexión exitosa")
        infoServer = conexion.get_server_info()
        print("Info del servidor: ", infoServer)
        
        cursor=conexion.cursor()
        cursor.execute("SELECT Canal, Año, Mes, CodArticulo, Actividad, Dia, NumeroTicket, Sucursal, Venta, Clasifica1, periodo FROM ventasdiario_c WHERE periodo in ('202212') and Clasifica1 not in ('Droguería','Droguerias','Droguerías');")
        data = cursor.fetchall()
        venta_diaria=pd.DataFrame(data)
        venta_diaria.columns= cursor.column_names
        
except Error as ex:
    print("Error durante conexión:", ex)
    
    
    

    


Conexión exitosa
Info del servidor:  5.7.21-log


In [144]:
dia = 15
venta_diaria = venta_diaria[venta_diaria["Dia"] <= dia]
pivot_venta_ = pd.pivot_table(venta_diaria,index = ["Año","Mes","Dia","Sucursal","Clasifica1"],values = ["Venta","NumeroTicket"],aggfunc = ["sum","count"])
pivot_venta_[("count", "venta/trx")] = pivot_venta_[("sum", "Venta")] /  pivot_venta_[("count", "NumeroTicket")]
venta_table =pivot_venta_.reset_index()
venta_table.columns = ["Año","Mes","Dia","Sucursal","Clasifica1","Venta","trx","Venta2","Venta / trx"]
venta_table = venta_table[["Año","Mes","Dia","Sucursal","Clasifica1","Venta","trx","Venta / trx"]]

etiqueta_mes = {"Mes_etq":{"Ene":1,
                           "Feb":2,
                           "Mar":3,
                           "Abr":4,
                           "May":5,
                           "Jun":6,
                           "Jul":7,
                           "Ago":8,
                           "Sep":9,
                           "Oct":10,
                           "Nov":11,
                           "Dic":12}}
    
venta_table["Mes_num"] = venta_table["Mes"].map(etiqueta_mes["Mes_etq"])

fechas_venta_list = []
for i in range(0,len(venta_table)):
    fecha_venta =   datetime.datetime(venta_table.iloc[i,0],venta_table.iloc[i,8], venta_table.iloc[i,2])
    fechas_venta_list.append(fecha_venta)

venta_table["Fecha"] = fechas_venta_list
venta_table["Fecha"] = pd.to_datetime(venta_table["Fecha"],format="%d/%m/%Y")

venta_table["Sucursal"] = pd.to_numeric(venta_table["Sucursal"], downcast = "integer")
 
nombre_archivo = anio + mes + " VentaDiaria.xlsx"
ruta = os.path.join(r"C:\Users\luis.montoya\OneDrive - Grupo Socofar\PowerBI - Última milla\Domifacil, Concentrador, Bopos, Ventas\Venta Diaria",nombre_archivo)

venta_table = venta_table[["Año","Mes","Dia","Sucursal","Clasifica1","Venta","trx","Venta / trx","Fecha"]]
venta_table["Sucursal"]  = pd.to_numeric(venta_table["Sucursal"], downcast = "integer")
venta_table.to_excel(ruta,index = False)

C:\Users\luis.montoya\AppData\Local\Temp\ipykernel_19380\1746996406.py:3: FutureWarning: pivot_table dropped a column because it failed to aggregate. This behavior is deprecated and will raise in a future version of pandas. Select only the columns that can be aggregated.
  pivot_venta_ = pd.pivot_table(venta_diaria,index = ["Año","Mes","Dia","Sucursal","Clasifica1"],values = ["Venta","NumeroTicket"],aggfunc = ["sum","count"])


### Cobro de domicilios

In [145]:
cursor.execute("SELECT Sucursal, NumeroTicket, CodArticulo, Venta, Ano, Mes, Dia, clasifica FROM ventasdiariogeoposh where periodo in ('202212') and clasifica = 'OMS';")
data = cursor.fetchall()
cobro_oms=pd.DataFrame(data)
cobro_oms.columns= cursor.column_names

cobro_oms = cobro_oms[cobro_oms["CodArticulo"]==297540]
cobro_oms =cobro_oms[['Sucursal','CodArticulo','Venta','Ano','Mes','Dia',"clasifica"]]

cobro_oms["Mes_num"] = cobro_oms["Mes"].map(etiqueta_mes["Mes_etq"])

fechas_cobro_list = []
for i in range(0,len(cobro_oms)):
    fecha_venta2 =   datetime.datetime(cobro_oms.iloc[i,3],cobro_oms.iloc[i,7], cobro_oms.iloc[i,5])
    fechas_cobro_list.append(fecha_venta2)

cobro_oms["Fecha_linea"] = fechas_cobro_list
cobro_oms["Fecha_linea"] = pd.to_datetime(cobro_oms["Fecha_linea"],format="%d/%m/%Y")

cobro_oms = cobro_oms[['Sucursal',"CodArticulo","Venta","clasifica","Fecha_linea"]]
cobro_oms = cobro_oms.reindex(columns=["Fecha_linea","Sucursal","CodArticulo","clasifica","Venta"])
cobro_oms.columns = ["Fecha_linea","Sucursal","Codigo","Canal","Valor"]


In [146]:
cobro_domifacil = pd.read_excel(r"C:\Users\luis.montoya\Downloads\Mostrador\CobroDomicilios\Sku_Domifacil.xlsx")
cobro_domifacil = cobro_domifacil[cobro_domifacil.SKU.isin([531595,531594,531543,122580])]
cobro_domifacil["Canal"] = "Domifacil"

etiqueta_cobro_domi = {"cobro_cod":{531595:3000,
                           531594:2500,
                           531543:1900,
                           122580:2500}}
    
cobro_domifacil["Valor"] = cobro_domifacil["SKU"].map(etiqueta_cobro_domi["cobro_cod"])

cobro_domifacil = cobro_domifacil[cobro_domifacil.Estado.isin(["processed","complete","processed_modified","in_street"])]

fechas_cobro =cobro_domifacil["Fecha creación"]
nuevas_fechas_cobro = []

for elemento in fechas_cobro:
    fecha_cobro_domi = datetime.datetime(elemento.year,elemento.month,elemento.day)
    nuevas_fechas_cobro.append(fecha_cobro_domi)

cobro_domifacil["Fecha_linea"] = nuevas_fechas_cobro

cobro_domifacil["Fecha_linea"] = pd.to_datetime(cobro_domifacil["Fecha_linea"],format = "%d/%m/%Y")
cobro_domifacil = cobro_domifacil[["Código Local","SKU","Canal","Valor","Fecha_linea"]]
cobro_domifacil = cobro_domifacil.reindex(columns=["Fecha_linea","Código Local","SKU","Canal","Valor"])
cobro_domifacil.columns = ["Fecha_linea","Sucursal","Codigo","Canal","Valor"]

In [147]:
cobro_mostrador = pd.read_csv(r"C:\Users\luis.montoya\Downloads\Mostrador\CobroDomicilios\Cobro Mostrador.csv",sep=",")

cobro_mostrador = cobro_mostrador[["Año","Mes","Día","Código Sucursal","Código Producto","Valor Total Línea"]]
cobro_mostrador["Canal"] = "Mostrador"
cobro_mostrador.columns = ["Año","Mes","Dia","Sucursal","Codigo","Valor","Canal"]

cobro_mostrador = cobro_mostrador[cobro_mostrador.Codigo.isin([531595,531594,531543,122580])]

fecha_cobro_mostrador = []
for i in range(0,len(cobro_mostrador)):
    fecha_linea = datetime.datetime(cobro_mostrador.iloc[i,0],cobro_mostrador.iloc[i,1],cobro_mostrador.iloc[i,2])
    fecha_cobro_mostrador.append(fecha_linea)

cobro_mostrador["Fecha_linea"] = fecha_cobro_mostrador
cobro_mostrador["Fecha_linea"] = pd.to_datetime(cobro_mostrador["Fecha_linea"],format = "%d/%m/%Y")

cobro_mostrador =cobro_mostrador[["Sucursal","Codigo","Valor","Canal","Fecha_linea"]]
cobro_mostrador = cobro_mostrador.reindex(columns=["Fecha_linea","Sucursal","Codigo","Canal","Valor"])
cobro_mostrador["Sucursal"] = pd.to_numeric(cobro_mostrador["Sucursal"],downcast = "integer") 

In [148]:
cobro_total = pd.concat([cobro_oms,cobro_domifacil,cobro_mostrador],axis = 0 )
dia_actual = date.today()
fecha_compara = datetime.datetime(dia_actual.year,dia_actual.month,dia_actual.day)
cobro_total=cobro_total[cobro_total["Fecha_linea"]<fecha_compara]
cobro_total["Sucursal"] = pd.to_numeric(cobro_total["Sucursal"],downcast="integer")

nombre_archivo = anio + mes + " CobroDomicilios.xlsx"
ruta = os.path.join(r"C:\Users\luis.montoya\OneDrive - Grupo Socofar\PowerBI - Última milla\CobroDomicilios",nombre_archivo)
cobro_total.to_excel(ruta,index=False,sheet_name = "Hoja1")

OSError: [Errno 22] Invalid argument: 'C:\\Users\\luis.montoya\\OneDrive - Grupo Socofar\\PowerBI - Última milla\\CobroDomicilios\\202212 CobroDomicilios.xlsx'